### class 1 _类一_
多线程对多个文字材料音频转换（每个线程负责一段）；输出文件与顺序有关

In [1]:
from openai import OpenAI
import dashscope
import time
import os 
import requests
import sys
import math 
import concurrent.futures
import tqdm
import threading
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
api_key = os.getenv("DASHSCOPE_API_KEY")

In [3]:
import dashscope.audio

In [ ]:
class MutiThreadASR:
    def __init__(self, filepath, num_process, savepath, VA):
        '''
        filepath: 包含文本的文本文件路径
        num_process: 线程数
        savepath: 保存路径'''
        self.filepath =filepath
        self.num_threads = num_threads
        self.savepath = savepath
        self.VA=VA
    def _get_files(self):
        '''
        返回包含行组成的列表'''
        file = self.filepath
        try:
            with open(file, 'r', encoding='utf-8')as f:
                paras = f.readlines() # list
            return paras
        except FileNotFoundError as e:
            print("{}".format(file)+e)
    def _load_one_part(self,chunk,worker_id):
        '''
        将包含行组成的列表转化为文本并调用text2voice'''
        # 将列表转化为文本
        text = "\n".join(chunk)  
        self._text2voice(text,worker_id)    
    def _get_chunk_size(self):
        '''
        返回每个线程应该读取的行数'''
        chunk_size=math.ceil(len(self._get_files())/self.num_threads)
        return chunk_size
    def _slice_file(self):
        '''
        返回包含行组成的列表的列表'''
        start=0
        reminder = len(self._get_files())%self._get_chunk_size()
        threads=[]
        for i in range(self.num_threads): # 应该用num_threads作为循环次数不是chunk_size
            end = start+self._get_chunk_size() +(1 if reminder>i else 0)
            threads.append(self._get_files()[start:end])
            start=end
        return threads
    def multi_threads(self):
        '''
        多线程处理,外部接口'''
        threads=[]
        sliced_lst=self._slice_file()
        
        for i, part in enumerate(sliced_lst):
            thread=threading.Thread(target=self._load_one_part, args=(part,i+1))
            thread.start()
            threads.append(thread)
        for thread in threads:
            thread.join()
    def _text2voice(self, text, worker_id):
        '''
        调用dashscope的api'''
        savepath = self.savepath+"part{}".format(worker_id)+"_{}".format(self.VA)+".mp3"
            
        # text is text not list
        response=dashscope.audio.qwen_tts.SpeechSynthesizer.call(
            model='qwen-tts',
            api_key=api_key,
            text=text,
            voice="Cherry",
        )
        audio_url=response.output.audio["url"]
        
        try:
            response=requests.get(audio_url)
            response.raise_for_status()
            with open(savepath,'wb')as f:
                f.write(response.content)
            print('音频文件保存到{}'.format(savepath))
        except Exception as e:
            print(e,'fail when downloading {}'.format(savepath))
                   
   

主函数

In [12]:
def main():
    filepath='flipped.txt'
    savepath='/Users/wtsama/Documents/code/'
    num_threads=4
    VA1="Ethan"
    VA2="Cherry"
    multiThreads2=MutiThreadASR(filepath,num_threads,savepath,VA=VA2)
    multiThreads2.multi_threads()
    multiThreads1=MutiThreadASR(filepath,num_threads,savepath,VA=VA1)
    multiThreads1.multi_threads()
    

In [13]:
if __name__=="__main__":
    main()

音频文件保存到/Users/wtsama/Documents/code/part1_Cherry.mp3
音频文件保存到/Users/wtsama/Documents/code/part3_Cherry.mp3
音频文件保存到/Users/wtsama/Documents/code/part4_Cherry.mp3
音频文件保存到/Users/wtsama/Documents/code/part2_Cherry.mp3
音频文件保存到/Users/wtsama/Documents/code/part3_Ethan.mp3
音频文件保存到/Users/wtsama/Documents/code/part1_Ethan.mp3
音频文件保存到/Users/wtsama/Documents/code/part4_Ethan.mp3
音频文件保存到/Users/wtsama/Documents/code/part2_Ethan.mp3
